📈 Amazon Financial AI Agent - Assessment Demo
This Notebook demonstrates the technical solution for Amazon (AMZN) financial analysis, integrating AWS Bedrock, LangGraph, and AWS Cognito. The entire execution is monitored using Langfuse Cloud.

🛠️ Observability Dashboard (Langfuse)
We begin by validating that the project is correctly connected to Langfuse for comprehensive trace tracking.

Image Reference: Dashboard General.png

In [14]:
import boto3
import os
import requests
import json
import hmac
import hashlib
import base64
import time
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

USER_POOL_ID = os.getenv("USER_POOL_ID")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("COGNITO_CLIENT_SECRET")
AWS_REGION = os.getenv("AWS_REGION", "us-east-2")
USERNAME = "test_user@example.com"
PASSWORD = "YourPassword123!"

def get_secret_hash(username, client_id, client_secret):
    message = username + client_id
    dig = hmac.new(str(client_secret).encode('utf-8'), 
                  msg=str(message).encode('utf-8'), 
                  digestmod=hashlib.sha256).digest()
    return base64.b64encode(dig).decode()

🔐 1. AWS Cognito Authentication
The backend requires a JWT Token to authorize queries. We utilize the USER_PASSWORD_AUTH flow for secure access.

In [15]:
access_token = None
client = boto3.client('cognito-idp', region_name=AWS_REGION)

try:
    s_hash = get_secret_hash(USERNAME, CLIENT_ID, CLIENT_SECRET)
    response = client.initiate_auth(
        ClientId=CLIENT_ID,
        AuthFlow='USER_PASSWORD_AUTH',
        AuthParameters={'USERNAME': USERNAME, 'PASSWORD': PASSWORD, 'SECRET_HASH': s_hash}
    )
    access_token = response['AuthenticationResult']['AccessToken']
    print("✅ Token successfully obtained.")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Token successfully obtained.


Identity Verification Evidence:

Image Reference: Auth de Cognito.png

Image Reference: Auth de Cognito BackEnd Python.png

In [16]:
def run_query(query):
    url = "http://localhost:8000/chat"
    headers = {"Authorization": f"Bearer {access_token}"}
    print(f"\n🔎 EXECUTING: {query}\n" + "="*70)
    
    with requests.post(url, json={"message": query}, headers=headers, stream=True) as r:
        for line in r.iter_lines():
            if line:
                chunk = line.decode('utf-8').replace("data: ", "")
                try:
                    data = json.loads(chunk)
                    if "agent" in data:
                        msg = data["agent"]["messages"][-1]
                        if isinstance(msg, str): print(f"🤖 AGENT: {msg}")
                        elif isinstance(msg, list):
                            for item in msg:
                                if item.get('type') == 'tool_use': 
                                    print(f"🛠️ TOOL CALL: {item['name']}")
                    elif "tools" in data:
                        print(f"📊 DATA RETRIEVED: {data['tools']['messages'][0]}")
                except: continue
    time.sleep(1)

⚡ 2. Assessment Test Battery
A. Real-time Market Data
Validating the agent's ability to fetch live stock prices.

In [17]:
run_query("What is the stock price for Amazon right now?")


🔎 EXECUTING: What is the stock price for Amazon right now?
🛠️ TOOL CALL: retrieve_realtime_stock_price
📊 DATA RETRIEVED: Latest Price for AMZN: $198.79
🤖 AGENT: As of right now (February 16, 2026), Amazon's stock (AMZN) is trading at $198.79 per share. This is the most recent closing price from the market. Would you like to see any historical price comparisons or learn more about Amazon's recent financial performance?


Langfuse Trace (Real-time Tooling):

Image Reference: 1. Precio en tiempo real.png

B. Temporal Reasoning (Historical Q4 2024)
Testing Temporal Grounding to identify the date range for Q4 2024.

In [18]:
run_query("What were the stock prices for Amazon in Q4 2024?")


🔎 EXECUTING: What were the stock prices for Amazon in Q4 2024?
🛠️ TOOL CALL: retrieve_historical_stock_price
📊 DATA RETRIEVED: {Timestamp('2024-10-01 00:00:00-0400', tz='America/New_York'): 185.1300048828125, Timestamp('2024-10-02 00:00:00-0400', tz='America/New_York'): 184.75999450683594, Timestamp('2024-10-03 00:00:00-0400', tz='America/New_York'): 181.9600067138672, Timestamp('2024-10-04 00:00:00-0400', tz='America/New_York'): 186.50999450683594, Timestamp('2024-10-07 00:00:00-0400', tz='America/New_York'): 180.8000030517578, Timestamp('2024-10-08 00:00:00-0400', tz='America/New_York'): 182.72000122070312, Timestamp('2024-10-09 00:00:00-0400', tz='America/New_York'): 185.1699981689453, Timestamp('2024-10-10 00:00:00-0400', tz='America/New_York'): 186.64999389648438, Timestamp('2024-10-11 00:00:00-0400', tz='America/New_York'): 188.82000732421875, Timestamp('2024-10-14 00:00:00-0400', tz='America/New_York'): 187.5399932861328, Timestamp('2024-10-15 00:00:00-0400', tz='America/New_Yo

Langfuse Trace (Historical Logic):

Image Reference: 2. Histórico Q4 2024.png

C. Performance vs Analyst Predictions
Comparing performance against report projections.

In [19]:
run_query("Compare Amazon's recent stock performance to analyst predictions in the reports.")


🔎 EXECUTING: Compare Amazon's recent stock performance to analyst predictions in the reports.
🛠️ TOOL CALL: retrieve_realtime_stock_price
📊 DATA RETRIEVED: Latest Price for AMZN: $198.79
🛠️ TOOL CALL: retrieve_historical_stock_price
📊 DATA RETRIEVED: {Timestamp('2025-11-17 00:00:00-0500', tz='America/New_York'): 232.8699951171875, Timestamp('2025-11-18 00:00:00-0500', tz='America/New_York'): 222.5500030517578, Timestamp('2025-11-19 00:00:00-0500', tz='America/New_York'): 222.69000244140625, Timestamp('2025-11-20 00:00:00-0500', tz='America/New_York'): 217.13999938964844, Timestamp('2025-11-21 00:00:00-0500', tz='America/New_York'): 220.69000244140625, Timestamp('2025-11-24 00:00:00-0500', tz='America/New_York'): 226.27999877929688, Timestamp('2025-11-25 00:00:00-0500', tz='America/New_York'): 229.6699981689453, Timestamp('2025-11-26 00:00:00-0500', tz='America/New_York'): 229.16000366210938, Timestamp('2025-11-28 00:00:00-0500', tz='America/New_York'): 233.22000122070312, Timestamp('2

Langfuse Trace (Analyst Comparison):

Image Reference: 3. Comparación vs Analistas.png

D. AI Innovation & AWS Business Growth
Extracting strategic insights from official reports.

In [20]:
run_query("Give me the current AMZN price and details about their AI and AWS growth.")


🔎 EXECUTING: Give me the current AMZN price and details about their AI and AWS growth.
🛠️ TOOL CALL: retrieve_realtime_stock_price
📊 DATA RETRIEVED: Latest Price for AMZN: $198.79
🛠️ TOOL CALL: query_amazon_reports
📊 DATA RETRIEVED: Fundamentally, if your mission is to make customers’ lives better and easier every day, and you believe every customer experience will be reinvented by AI, you’re going to invest deeply and broadly in AI. That’s why there are more than 1,000 GenAI applications being built across Amazon, aiming to meaningfully change customer experiences in shopping, coding, personal assistants, streaming video and music, advertising, healthcare, reading, and home devices, to name a few. It’s also why AWS is quickly developing the key primitives (or building blocks) for AI development, such as custom silicon AI chips in Amazon Trainium to provide better price-performance on training and inference, highly flexible model-building and inference services in Amazon SageMaker and

Langfuse Trace (Strategic RAG):

Image Reference: 4. Negocio de IA y AWS.png

E. Physical Infrastructure (Office Space)
Precision RAG query for North American office ownership.

In [21]:
run_query("How much office space did Amazon own in North America in 2024?")


🔎 EXECUTING: How much office space did Amazon own in North America in 2024?
🛠️ TOOL CALL: query_amazon_reports
📊 DATA RETRIEVED: Segment Leased Square     Footage (1) Owned Square     Footage (1)     North America 457,104 20,741 International 166,673 11,741 AWS 24,875 24,052 Total 648,652 56,534      ___________________ (1) Segment amounts exclude corporate facilities. Shared facilities are allocated among the segments based on usage and     primarily relate to facilities that hold our technology infrastructure. See Item 8 of Part II, “Financial Statements and Supplementary Data — Note 10 — Segment Information.”     We own and lease our corporate headquarters in Washington’s Puget Sound region and Arlington, Virginia.     Item 3. Legal Proceedings     See Item 8 of Part II, “Financial Statements and Supplementary Data — Note 7 — Commitments and Contingencies — Legal Proceedings.”     Item 4. Mine Safety Disclosures     Not applicable.     18PART II     Item 5. Market for the Registran

Langfuse Trace (RAG Accuracy):

Image Reference: 5. Datos de Oficina (RAG).png

🎯 Conclusion
The AI Agent solution successfully fulfills all enterprise-grade requirements:

Secure Access: Integrated with AWS Cognito.

Dynamic Reasoning: Leverages LangGraph.

Data Accuracy: Combines live APIs and Bedrock Knowledge Bases.

Total Transparency: Audited 100% via Langfuse.